In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import argparse
import noise
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import MotionClouds as mc
from torchvision import datasets, transforms
from torch.autograd import Variable
from LogGabor import LogGabor

## Charger la matrice de certitude

In [2]:
path = "MNIST_accuracy.npy"
if os.path.isfile(path):
    print('Loading accuracy...')
    accuracy =  np.load(path)
    print(accuracy)
else:
    print('No accuracy data found.')

Loading accuracy...
[[0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 ...
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]]


## Préparer l'apprentissage et les fonctions nécessaires au fonctionnement du script

In [3]:
def vectorization(N_theta, N_orient, N_scale, N_phase, N_X, N_Y, rho):
    phi = np.zeros((N_theta, N_orient, N_scale, N_phase, N_X*N_Y))
    parameterfile = 'https://raw.githubusercontent.com/bicv/LogGabor/master/default_param.py'
    lg = LogGabor(parameterfile)
    lg.set_size((N_X, N_Y))
    params= {'sf_0':.1, 'B_sf': lg.pe.B_sf, 'theta':np.pi* 5 / 7., 'B_theta': lg.pe.B_theta}
    phase = np.pi/4
    edge = lg.normalize(lg.invert(lg.loggabor(N_X/3, 3*N_Y/4, **params)*np.exp(-1j*phase)))
    
    for i_theta in range(N_theta):
        for i_orient in range(N_orient):
            for i_scale in range(N_scale):
                ecc =  (1/rho)**(N_scale - i_scale)
                r = np.sqrt(N_X**2+N_Y**2) / 2 * ecc # radius
                sf_0 = 0.5 * 0.03 / ecc
                x = N_X/2 + r * np.cos((i_orient+(i_scale % 2)*.5)*np.pi*2 / N_orient)
                y = N_Y/2 + r * np.sin((i_orient+(i_scale % 2)*.5)*np.pi*2 / N_orient)            
                for i_phase in range(N_phase):
                    params= {'sf_0':sf_0, 'B_sf': lg.pe.B_sf, 'theta':i_theta*np.pi/N_theta, 'B_theta': np.pi/N_theta/2}
                    phase = i_phase * np.pi/2
                    phi[i_theta, i_orient, i_scale, i_phase, :] = lg.normalize(lg.invert(lg.loggabor(x, y, **params)*np.exp(-1j*phase))).ravel()            
    return phi

N_theta, N_orient, N_scale, N_phase, N_X, N_Y, rho = 6, 12, 5, 2, 128, 128, 1.61803
phi = vectorization(N_theta, N_orient, N_scale, N_phase, N_X, N_Y, rho)
phi_vector = phi.reshape((N_theta*N_orient*N_scale*N_phase, N_X*N_Y))
phi_plus = np.linalg.pinv(phi_vector)

energy = (phi**2).sum(axis=(0,3)) 
energy /= energy.sum(axis=-1)[:, :, None]
energy_vector = energy.reshape((N_orient*N_scale, N_X*N_Y))
energy_plus = np.linalg.pinv(energy_vector)

def accuracy_128(i_offset, j_offset, N_pic=128, N_stim=55):
    center = (N_pic-N_stim)//2
    
    accuracy_128 = 0.1 * np.ones((N_pic,N_pic))
    accuracy_128[(center+i_offset):(center+N_stim+i_offset),(center+j_offset):(center+N_stim+j_offset)] = accuracy
    
    accuracy_LP = energy_vector @ np.ravel(accuracy_128)    
    return accuracy_LP

def mnist_128(data, i_offset, j_offset, N_pic=128, N_stim=28, noise=True, noise_type='MotionCloud'):
    center = (N_pic-N_stim)//2
    
    data_128 = data.min() * np.ones((N_pic,N_pic))
    data_128[(center+i_offset):(center+N_stim+i_offset),(center+j_offset):(center+N_stim+j_offset)] = data
    
    if noise:
        if noise_type == 'MotionCloud':
            data_LP = phi_vector @ np.ravel(data_128 + MotionCloudNoise())
        elif noise_type == 'Perlin':
            data_LP = phi_vector @ np.ravel(data_128 + randomized_perlin_noise())     
    else:
        data_LP = phi_vector @ np.ravel(data_128)     
    return data_LP

def couples(data, i_offset, j_offset):
    v = mnist_128(data, i_offset, j_offset)
    a = accuracy_128(i_offset, j_offset)    
    return (v, a)

def minmax(value, border):
    value = max(value, -border)
    value = min(value, border)
    return value

def sigmoid(values):
    values = 1 / (1 + ((1 / 0.1) - 1) * np.exp(-values))
    return values

def randomized_perlin_noise(shape=(128,128), scale=10, octaves=6, persistence=0.5, lacunarity=2.0, base=0):
    noise_vector = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            noise_vector[i][j] = noise.pnoise2(i/scale,
                                        j/scale,
                                        octaves=int(octave * abs(np.random.randn()))+1,
                                        persistence=persistence * abs(np.random.randn()),
                                        lacunarity=lacunarity * abs(np.random.randn()),
                                        repeatx=shape[0], 
                                        repeaty=shape[1], 
                                        base=base)           
    return noise_vector

def MotionCloudNoise(sf_0=0.125, B_sf=3.):
    mc.N_X, mc.N_Y, mc.N_frame = 128, 128, 1
    fx, fy, ft = mc.get_grids(mc.N_X, mc.N_Y, mc.N_frame)
    name = 'static'
    env = mc.envelope_gabor(fx, fy, ft, sf_0=sf_0, B_sf=B_sf, B_theta=np.inf, V_X=0., V_Y=0., B_V=0, alpha=.5)
    
    z = mc.rectif(mc.random_cloud(env))
    z = z.reshape((mc.N_X, mc.N_Y))
    return z

In [8]:
sample_size = 100 #quantity of examples that'll be processed

data_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', 
                   train=True,    #def the dataset as training data 
                   download=True, #download if dataset not present on disk
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=sample_size, 
                   shuffle=True)

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden1, n_hidden2, n_output):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden1)
        self.hidden2 = torch.nn.Linear(n_hidden1, n_hidden2)
        self.predict = torch.nn.Linear(n_hidden2, n_output)
        
    def forward(self, data):
        data = F.relu(self.hidden1(data))
        data = self.hidden2(data)
        data = self.predict(data)
        data = F.sigmoid(data)
        return data
    
net = Net(n_feature=720, n_hidden1=585, n_hidden2=195, n_output=60)
optimizer = torch.optim.SGD(net.parameters(), lr=0.3)
loss_func = torch.nn.MSELoss()
#loss_func = torch.nn.CrossEntropyLoss()

def train(sample_size, vsize=720, asize=60):
    t_start = time.time()
    print('Starting training...')
    for batch_idx, (data, label) in enumerate(data_loader):
        data_v, data_a = np.zeros((sample_size, 1, vsize)), np.zeros((sample_size, 1, asize))
        for idx in range(sample_size):
            i_offset, j_offset = int(minmax(np.random.randn()*5, 35)), int(minmax(np.random.randn()*5, 35))
            data_v[idx,0,:], data_a[idx,0,:] = couples(data[idx,0,:], i_offset, j_offset)

        data_v, data_a = Variable(torch.FloatTensor(data_v)), Variable(torch.FloatTensor(data_a))
        print(data_v.shape)

        prediction = net(data_v)
        loss = loss_func(prediction, data_a)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Epoch {}: [{}/{}] Loss: {} Time: {:.2f} mn'.format(
                epoch, batch_idx*sample_size, len(data_loader.dataset), 
                loss.data.numpy(), (time.time()-t_start)/60))
    
def eval_one(vsize=720, asize=60, N_pic=128):
    print('Starting evaluation...')
    for batch_idx, (data, label) in enumerate(data_loader):
        data_v, data_a = np.zeros((1, 1, vsize)), np.zeros((1, 1, asize))

        i_offset, j_offset = int(minmax(np.random.randn()*5, 35)), int(minmax(np.random.randn()*5, 35))
        print('i_offset: {}, j_offset: {}'.format(i_offset, j_offset))
        data_v[0,0,:], data_a[0,0,:] = couples(data[0,0,:], i_offset, j_offset)

        data_v, data_a = Variable(torch.FloatTensor(data_v)), Variable(torch.FloatTensor(data_a))

        prediction = net(data_v)        
        pred_data = prediction.data.numpy()[-1][-1]

        image = sigmoid(energy_plus @ pred_data)
        image_reshaped = image.reshape(N_pic,N_pic)
        
        fig, ax = plt.subplots(figsize=(13,10.725))
        cmap = ax.pcolor(np.arange(-(N_pic/2),(N_pic/2)), np.arange(-(N_pic/2),(N_pic/2)), image_reshaped)
        fig.colorbar(cmap)
        plt.axvline(j_offset, c='k'); plt.axhline(i_offset, c='k')
        
        for i_pred in range(0,N_pic):
            for j_pred in range(0,N_pic):
                if image_reshaped[i_pred][j_pred] == image_reshaped.max():
                    print(i_pred-(N_pic/2), j_pred-(N_pic/2))
                    plt.axvline(j_pred-(N_pic/2), c='r'); plt.axhline(i_pred-(N_pic/2), c='r')
                    break
        
        print('...Evaluation done')
        break

### Lancer l'apprentissage ou charger les données

In [9]:
path = 'regression_couples_noised.pt'

if os.path.isfile(path):
    print('Loading file', path)
    net.load_state_dict(torch.load(path))
else:
    print('Training model...')
    for epoch in range(5):             #number of training epochs
        train(epoch, sample_size)       #execute the learning functions
        torch.save(net.state_dict(), path) #save the neural network state
        print('Model saved at', path)

Training model...
Starting training...
torch.Size([0])


RuntimeError: cannot unsqueeze empty tensor

In [ ]:
epoch, sample_size

### Lancer l'évaluation

In [ ]:
eval_one()